In [ ]:
import numpy as np
import pickle
import cv2
import os
from glob import glob
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
from keras import backend as K
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [ ]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU not found')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [ ]:
def cnn_model():
    model = Sequential()
    model.add(Conv2D(16, (2, 2), input_shape=(30, 1662, 1), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(3, 3), padding='same'))
    model.add(Conv2D(64, (5, 5), activation='relu'))
    model.add(MaxPooling2D(pool_size=(5, 5), strides=(5, 5), padding='same'))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(50, activation='softmax'))
    sgd = optimizers.SGD(lr=1e-2)
    model.compile(loss='categorical_crossentropy',
                  optimizer=sgd, metrics=['accuracy'])
    filepath = "cnn_model_keras2.h5"
    checkpoint1 = ModelCheckpoint(
        filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
    callbacks_list = [checkpoint1]
    return model, callbacks_list

In [ ]:
def train():
    X_train, X_test, y_train, y_test = train_test_split(np.load('/content/drive/MyDrive/X.npy'), np_utils.to_categorical(np.load('/content/drive/MyDrive/y.npy')).astype(int), test_size=0.10)
    print(y_test.shape)

    model, callbacks_list = cnn_model()
    model.summary()
    model.fit(X_train, y_train, validation_data=(
        X_test, y_test), epochs=2000, batch_size=10, steps_per_epoch = 1823, callbacks=callbacks_list)
    scores = model.evaluate(X_test, y_test, verbose=0)
    print("CNN Error: %.2f%%" % (100-scores[1]*100))

In [ ]:
train()